**Script: "RIDER Balancing & Randomization of Conditions"**
**Project: RIDER**

Input: Image names in PNG format<br>
Output: Condition files (as csv files) that balance the appearance of these images across conditions to ensure that observed effects don't stem from image identity but from the conditions themselves<br>

Author: Frieda Born<br>
Contact: born (at) mpib-berlin.mpg.de<br>

This script is an attempt to improve the balancing stragtegy introsuced in the first <RIDER_balancing> script. To banalce the conditions below we will use a strictly latin square procedure and rotate the stimuli across all task conditions. We again have the following conditions: 
> Trial type 1, Test 1<br>
> Trial type 2, Test 1<br>
> Trial type 2, Test 2<br>
> Trial type 3, Test 1<br>
> Trial type 3, Test 2 (invisible, as not used in WM part of experiment)<br>

In [1698]:
# Environment preparation -------------------------------------------------
import os
import numpy as np
from typing import Any, List, Dict, Tuple
import pandas as pd
import random
import matplotlib.pyplot as plt
import copy
from collections import Counter
from collections import defaultdict
from operator import itemgetter
from itertools import groupby
import operator
from pprint import pprint

In [1699]:
# type-setting
Array = np.ndarray

In [1697]:
%reset

Once deleted, variables cannot be recovered. Proceed (y/[n])? y


In [1700]:
# define absolute path
path = "/Users/born/Documents/Upside_down_task/client/public/stimuli"

# we only want png file
stimuli_name_list = [el for el in os.listdir(path) if os.path.isfile(os.path.join(path, el)) if not(el.endswith("white.png")) and el.endswith('.png')]#one image is called white and cannot be used as a stimulus.
print(type(stimuli_name_list))
n_images = len(stimuli_name_list)
print("There are", n_images, "images in my image folder right now.")
stimuli_name_list

<class 'list'>
There are 110 images in my image folder right now.


['pylon01.png',
 'redfox.png',
 'soapdispenser03b.png',
 'stackingring01b.png',
 'desktopcomputer.png',
 'chessknight03b.png',
 'boxingglove01.png',
 'honeybee.png',
 'jeep.png',
 'nyala.png',
 'wheelbarrow01.png',
 'toyhockeystick02a.png',
 'pintofbeer.png',
 'radio03a.png',
 'graduationcap.png',
 'razor04a.png',
 'flipflop01a.png',
 'sailboat.png',
 'rollerblade.png',
 'dragonfly01.png',
 'tabascosauce.png',
 'fryingpan01.png',
 'wateringcan.png',
 'saw02b.png',
 'motoroilbottle04.png',
 'hat04a.png',
 'flamingo.png',
 'bluejay.png',
 'witchhat.png',
 'lynx01.png',
 'mailbox02.png',
 'lock02a.png',
 'zebra.png',
 'armadillo.png',
 'lightbulb01.png',
 'handfan01b.png',
 'thumbtack02b.png',
 'duck01.png',
 'candelabra.png',
 'handbrush02.png',
 'gorilla.png',
 'clownfish.png',
 'paintroller02.png',
 'shoppingcart.png',
 'acousticguitar02.png',
 'snowglobe.png',
 'officechair02.png',
 'windsurfboard.png',
 'outdoorheater.png',
 'megaphone.png',
 'buoy.png',
 'peacock.png',
 'hen.png',
 

In [1701]:
# I want the item names to be preceeded by "stimuli/", which makes it easier to use them as excel file later on
string = 'stimuli/'
stimuli_names = [string + x for x in stimuli_name_list]
stimuli_names[0]

'stimuli/pylon01.png'

In [1702]:
def sample_stimuli(stimuli_names: List[str], num_stimuli: int) -> List[str]:
    """Sample <num_stimuli> stimuli from the set of all stimuli without replacement."""
    sampled_stimuli = np.random.choice(a=stimuli_names, size=num_stimuli, replace=False).tolist()
    assert len(set(sampled_stimuli)) == len(sampled_stimuli), '\nStimuli must be unique.\n'
    return sampled_stimuli

In [1703]:
def add_orientations_to_stimuli(sampled_stimuli: List[str], orientations: List[str]):
    """Add the orientation directly to the string of the stimuli to enable fixed connection between stimuli and orientations"""
    stimuli_and_orientations = list()
    for stim in sampled_stimuli:
        orientation = np.random.choice(a=possible_orientations, size=1, replace=False)
        sampled_stimuli = [stim + str(orientation)]
        stimuli_and_orientations.append(sampled_stimuli)
    return stimuli_and_orientations

In [1704]:
def remove_stimuli(stimuli: List[str], sampled_stimuli: List[str]) -> None:
    """Remove stimuli that were sampled for a single trial from the set of possible exp. stimuli."""
    for stimulus in sampled_stimuli:
        stimuli.pop(stimuli.index(stimulus))

In [1705]:
def partition_stimuli_into_experimental_conditions(
    stimuli: List[str],
    num_conditions: int,
    num_trials_per_condition: int
) -> Dict[str, List[str]]:
    """Partition stimuli into the five different conditions."""
    condition_partitioning = {}
    stimuli_copy = copy.deepcopy(stimuli)
    for i in range(1, num_conditions+1):
        # one stimulus is needed per condition
        num_stimuli = num_trials_per_condition
        condition_stimuli = np.random.choice(a=stimuli_copy, size=num_stimuli, replace=False).tolist()
        remove_stimuli(stimuli=stimuli_copy, sampled_stimuli=condition_stimuli)
        condition_partitioning[f'condition{i}'] = condition_stimuli
    print(len(stimuli))
    return condition_partitioning

In [1706]:
def rotate_stimuli(
    condition_partitioning: Dict[str, List[str]],
    num_rotation_stimuli: int
) -> Dict[str, List[str]]:
    """Rotate stimuli through conditions by <num_rotation_stimuli>."""
    condition1_stimuli = condition_partitioning['condition1']
    condition2_stimuli = condition_partitioning['condition2']
    condition3_stimuli = condition_partitioning['condition3']
    condition4_stimuli = condition_partitioning['condition4']
    condition5_stimuli = condition_partitioning['condition5']
    condition_partitioning['condition1'] = condition5_stimuli[:num_rotation_stimuli]
    condition_partitioning['condition2'] = condition1_stimuli[:num_rotation_stimuli]
    condition_partitioning['condition3'] = condition2_stimuli[:num_rotation_stimuli]
    condition_partitioning['condition4'] = condition3_stimuli[:num_rotation_stimuli]
    condition_partitioning['condition5'] = condition4_stimuli[:num_rotation_stimuli]
    return condition_partitioning

In [1707]:
def sample_condition(stimuli: List[str], experiment_condition: str) -> Dict[str, str]:
    """Sample a single condition."""
    # initialize an empty dictionary to store all variables necessary for a single trial
    condition = {}
    if(experiment_condition == 'condition1'):
        # initialize a placeholder variable for 'image2' and 'test2'
        placeholder = '_'
        # draw a single stimulus for presenting images and testing retrieval 
        sampled_stimuli = random.choices(stimuli, k=1)
        condition['image1'] = sampled_stimuli[0]
        condition['image2'] = placeholder
        condition['test1']  = sampled_stimuli[0]
        condition['test2']  = placeholder
        condition['baselinePresent'] = '1'
        condition['baselinePresent2'] = '0'
        condition['adjust2Present'] = '0'
        condition['trial_type'] = 'baseline'
    elif(experiment_condition == 'condition2'):
        # draw one stimulus from the set of experimental stimuli without replacement
        sampled_stimuli = random.choices(stimuli, k=1)
        condition['image1'] = sampled_stimuli[0]
        condition['test1']  = sampled_stimuli[0]
        condition['baselinePresent'] = '0'
        condition['baselinePresent2'] = '1'
        condition['adjust2Present'] = '1'
        condition['trial_type'] = 't2'
    elif(experiment_condition == 'condition3'):
        # draw one stimulus from the set of experimental stimuli without replacement
        sampled_stimuli = random.choices(stimuli, k=1)
        condition['image2'] = sampled_stimuli[0]
        condition['test2']  = sampled_stimuli[0]
        condition['trial_type'] = 't2'
    elif(experiment_condition == 'condition4'):
        placeholder = '_'
        # draw one stimulus from the set of experimental stimuli without replacement
        sampled_stimuli = random.choices(stimuli, k=1)
        # randomly permute the order of the two stimuli for presenting images to participants
        condition['image1'] = sampled_stimuli[0]
        # randomly select one stimulus that is shown as image2
        condition['test1'] = sampled_stimuli[0]
        condition['test2'] = placeholder
        condition['baselinePresent'] = '0'
        condition['baselinePresent2'] = '1'
        condition['adjust2Present'] = '0'
        condition['trial_type'] = 't3'
    elif(experiment_condition == 'condition5'):
        sampled_stimuli = random.choices(stimuli, k=1)
        condition['image2'] = sampled_stimuli[0]
        condition['trial_type'] = 't3'
        # note that here is nothing following for condition5 because these stimuli are not shown in the WM experiment
        
    else:
        raise ValueError('\nThere are 5 conditions.\nUse condition1,condition2,condition3,condition4 or condition5 .\n')
    # remove sampled stimuli from the participant's set of stimuli so that they are not drawn again
    remove_stimuli(stimuli=stimuli, sampled_stimuli=sampled_stimuli)
    return condition

In [1708]:
def sample_conditions(
    condition_order: List[str], 
    condition_sets: Dict[str, List[str]],
) -> List[str]:
    """Sample stimuli for all trials in a practice or experimental session."""
    sampled_conditions = [sample_condition(condition_sets[condition], condition) for condition in condition_order]
    return sampled_conditions

In [1709]:
def participant_randomization(
    sessions: Dict[str, int],
    condition_sets: Dict[str, List[str]],
    conditions: List[str],
    num_conditions: int,
) -> List[str]:
    """Sample practice and experimental conditions for a single participant."""
    participant_conditions = []
    for session, number_conditions in sessions.items():
        # number of conditions per type
        num_conditions_per_type = number_conditions // len(conditions)
        # extend the list of condition types by <num_conditions_per_type>
        condition_order = conditions * num_conditions_per_type
        # randomly permute the order of condition types for a single session
        random.shuffle(condition_order)
        sampled_conditions = sample_conditions(condition_order,condition_sets)
        participant_conditions.extend(sampled_conditions)        
    #participant_conditions = pd.DataFrame(participant_conditions)
    print(num_conditions_per_type)
    print(len(conditions))
    print(number_conditions)
    print(condition_order)
    return participant_conditions

In [314]:
#participant_conditions = participant_randomization(sessions = sessions, condition_sets = partitioning, conditions = conditions,num_conditions=4)


In [1710]:
def stimuli_balancing(
    num_participants: int,
    sessions: Dict[str, int],
    condition_partitioning: Dict[str, List[str]],
    conditions: List[str],
) -> List[str]:
    """
    This is the main loop for balancing the experimental conditions. 
    Here, we sample both practice and experimental conditions for all participants.
    """
    num_conditions = len(conditions)
    particpant_randomizations = []
    for i in range(1, num_participants+1):
        if i > 1:
            # rotate stimuli through the three type sets by <num_rotation_stimuli> (currently 22)
            condition_partitioning = rotate_stimuli(condition_partitioning, num_rotation_stimuli = 22)
        # create copies of the three trial type sets so that we can remove stimuli 
        # from the three trial type sets without changing the original sets
        condition_partitioning_copy = copy.deepcopy(condition_partitioning)
        # sample practice and experimental trials for a single participant
        participant_conditions = participant_randomization(
            sessions=sessions,
            condition_sets=condition_partitioning_copy,
            conditions=conditions,
            num_conditions=num_conditions,
        )
        # add start orientations to conditions file
        #participant_trials['startOri1'] = start_orientations[:, 0]
        #participant_trials['startOri2'] = start_orientations[:, 1]
        #participant_trials['testOri1']  = start_orientations[:, 2]
        #participant_trials['testOri2']  = start_orientations[:, 3]
        
        # permute the order of the presented stimuli (i.e., shuffle rows) to counteract potential orientation biases
        #participant_conditions = participant_conditions.sample(frac=1, replace=False).reset_index(drop=True)
        particpant_randomizations.append(participant_conditions)
    return particpant_randomizations

In [1711]:
# seed random number generator
rnd_seed = 42
random.seed(rnd_seed)
np.random.seed(rnd_seed)

In [1712]:
#### define experimental variables ####
num_stimuli = 110
num_practice_stimuli = 10
num_exp_stimuli = num_stimuli - num_practice_stimuli
num_participants = 5

ttypes = ['t1', 't2', 't3']
conditions = ['condition1','condition2','condition3','condition4', 'condition5']
num_practice_conditions = 0
num_exp_conditions = 110
num_total_conditions = num_practice_conditions + num_exp_conditions
sessions = {'practice': num_practice_conditions, 'experimental': num_exp_conditions}

stimulus_order = ['stim1','stim2','stim3','stim4','stim5']

# compute number of trials per trial type
num_condition_per_type =  22# num_total_trials // len(ttypes)

# define variables for orientations of the stimuli
min_orientation = 11.25
max_orientation = 371.25
step_size =  22.5

possible_orientations = np.arange(min_orientation, max_orientation, step_size)
possible_orientations = possible_orientations.tolist()
possible_orientations = possible_orientations * 7
len(possible_orientations)

# np.rint(np.linspace(min_orientation, max_orientation, num_orientations))
#possible_orientations = np.linspace(min_orientation, max_orientation, num_orientations).round(1)

112

In [1713]:
# sample stimuli without replacement
sampled_stimuli = sample_stimuli(stimuli_names=stimuli_names, num_stimuli=num_stimuli)
sampled_stimuli

['stimuli/makeupbrush04.png',
 'stimuli/wheelbarrow01.png',
 'stimuli/desktopcomputer.png',
 'stimuli/fireextinguisher02.png',
 'stimuli/eiffeltower.png',
 'stimuli/scissors02a.png',
 'stimuli/mailbox02.png',
 'stimuli/snowglobe.png',
 'stimuli/dromedary.png',
 'stimuli/toyhockeystick02a.png',
 'stimuli/boot01.png',
 'stimuli/needlenosepliers03b.png',
 'stimuli/pylon01.png',
 'stimuli/comb02b.png',
 'stimuli/rollerblade.png',
 'stimuli/parrot01.png',
 'stimuli/accordion02.png',
 'stimuli/globe.png',
 'stimuli/cornet.png',
 'stimuli/paintroller02.png',
 'stimuli/pintofbeer.png',
 'stimuli/thumbtack02b.png',
 'stimuli/cat.png',
 'stimuli/flamingo.png',
 'stimuli/wateringcan.png',
 'stimuli/lock02a.png',
 'stimuli/windsurfboard.png',
 'stimuli/saintbernard.png',
 'stimuli/razor04a.png',
 'stimuli/acousticguitar02.png',
 'stimuli/bicycle.png',
 'stimuli/gecko.png',
 'stimuli/nyala.png',
 'stimuli/armadillo.png',
 'stimuli/gardengnome01.png',
 'stimuli/pear01.png',
 'stimuli/witchhat.png',


In [1714]:
# add orientations to stimuli that can be extracted to independant variables later
sampled_stimuli = add_orientations_to_stimuli(sampled_stimuli = sampled_stimuli, orientations = possible_orientations)
# flatten List of Lists Using a List Comprehension
sampled_stimuli = [item for sublist in sampled_stimuli for item in sublist]

In [1715]:
sampled_stimuli

['stimuli/makeupbrush04.png[326.25]',
 'stimuli/wheelbarrow01.png[11.25]',
 'stimuli/desktopcomputer.png[281.25]',
 'stimuli/fireextinguisher02.png[326.25]',
 'stimuli/eiffeltower.png[168.75]',
 'stimuli/scissors02a.png[56.25]',
 'stimuli/mailbox02.png[101.25]',
 'stimuli/snowglobe.png[146.25]',
 'stimuli/dromedary.png[303.75]',
 'stimuli/toyhockeystick02a.png[258.75]',
 'stimuli/boot01.png[191.25]',
 'stimuli/needlenosepliers03b.png[191.25]',
 'stimuli/pylon01.png[213.75]',
 'stimuli/comb02b.png[258.75]',
 'stimuli/rollerblade.png[146.25]',
 'stimuli/parrot01.png[56.25]',
 'stimuli/accordion02.png[78.75]',
 'stimuli/globe.png[168.75]',
 'stimuli/cornet.png[236.25]',
 'stimuli/paintroller02.png[303.75]',
 'stimuli/pintofbeer.png[281.25]',
 'stimuli/thumbtack02b.png[56.25]',
 'stimuli/cat.png[146.25]',
 'stimuli/flamingo.png[213.75]',
 'stimuli/wateringcan.png[258.75]',
 'stimuli/lock02a.png[281.25]',
 'stimuli/windsurfboard.png[348.75]',
 'stimuli/saintbernard.png[213.75]',
 'stimuli/r

In [1716]:
# partition the stimuli into three disjoint sets, one for each trial type
condition_partitioning = partition_stimuli_into_experimental_conditions(
    stimuli=sampled_stimuli,
# we need to partition the stimuli across all 5 conditions, although trial type 3 test 2 will be "invisible"
    num_conditions= 5,
    num_trials_per_condition=22
)
condition_partitioning

110


{'condition1': ['stimuli/fryingpan01.png[168.75]',
  'stimuli/saw02b.png[303.75]',
  'stimuli/dromedary.png[303.75]',
  'stimuli/gecko.png[326.25]',
  'stimuli/clownfish.png[348.75]',
  'stimuli/handfan01b.png[78.75]',
  'stimuli/chisel01b.png[146.25]',
  'stimuli/acousticguitar02.png[236.25]',
  'stimuli/bulldozer.png[303.75]',
  'stimuli/motorcycle.png[146.25]',
  'stimuli/quartz.png[11.25]',
  'stimuli/windsurfboard.png[348.75]',
  'stimuli/nail.png[56.25]',
  'stimuli/outdoorheater.png[213.75]',
  'stimuli/chessknight03b.png[326.25]',
  'stimuli/photocopier.png[303.75]',
  'stimuli/loppingshears.png[213.75]',
  'stimuli/travelmug.png[56.25]',
  'stimuli/handbrush02.png[56.25]',
  'stimuli/eiffeltower.png[168.75]',
  'stimuli/maraca02a.png[303.75]',
  'stimuli/officechair02.png[348.75]'],
 'condition2': ['stimuli/hat04a.png[168.75]',
  'stimuli/acorn.png[56.25]',
  'stimuli/armadillo.png[213.75]',
  'stimuli/witchhat.png[146.25]',
  'stimuli/zebra.png[348.75]',
  'stimuli/blender.pn

In [1717]:
# get all participant condition randomizations
particpant_randomizations = stimuli_balancing(
    num_participants=5,
    sessions=sessions, 
    condition_partitioning= condition_partitioning,
    conditions=conditions, 
)


22
5
110
['condition2', 'condition5', 'condition2', 'condition1', 'condition5', 'condition1', 'condition2', 'condition4', 'condition2', 'condition2', 'condition5', 'condition1', 'condition1', 'condition3', 'condition4', 'condition4', 'condition3', 'condition5', 'condition1', 'condition3', 'condition5', 'condition3', 'condition1', 'condition3', 'condition4', 'condition1', 'condition3', 'condition4', 'condition2', 'condition3', 'condition3', 'condition4', 'condition1', 'condition2', 'condition5', 'condition3', 'condition5', 'condition2', 'condition2', 'condition5', 'condition1', 'condition4', 'condition3', 'condition5', 'condition4', 'condition4', 'condition1', 'condition2', 'condition4', 'condition5', 'condition1', 'condition4', 'condition3', 'condition1', 'condition5', 'condition3', 'condition5', 'condition5', 'condition2', 'condition3', 'condition2', 'condition2', 'condition4', 'condition5', 'condition1', 'condition3', 'condition4', 'condition2', 'condition1', 'condition4', 'condition

In [1718]:
# Sort conditions in trial types
def sort_conditions_in_trial_types(
    num_participants: int,
    particpant_randomizations: List[str],
)-> List[str]:
    """Sort conditions in trial type 1 (baseline) and 3"""
    trial_types = itemgetter('trial_type')
    participant_grouped_conditions = {}
    # outer loop: loop over each randomization
    for i, randomization in enumerate(particpant_randomizations, start=1):
        trial_type_grouping = defaultdict(list)
         # inner loop: loop over every condition in current subject's randomization
        for condition in randomization:
            trial_type_grouping[trial_types(condition)].append(condition)
        participant_grouped_conditions[f'participant_{i:02d}'] = trial_type_grouping
        
    """
    trial_types = itemgetter('trial_type')
    trial_type_grouping = defaultdict(lambda: defaultdict(list))
    for i, randomization in enumerate(particpant_randomizations, start=1):
        for condition in randomization:
            trial_type_grouping[f'participant_{i:02d}'][trial_types(condition)].append(condition)
    """
    return participant_grouped_conditions

In [1719]:
sorted_conditions = sort_conditions_in_trial_types(num_participants = 5 ,particpant_randomizations=particpant_randomizations)


In [1782]:
# From here it goes participant per participant
baseline_trials       = sorted_conditions['participant_05']['baseline']
t3_trials             = sorted_conditions['participant_05']['t3']
t2_trials             = sorted_conditions['participant_05']['t2']

# to construct the final trials I need to extract the trial variables that I need from the conditions
test2_stimuli      = [stimulus['test2'] for stimulus in t2_trials if 'test2' in stimulus]
test1_stimuli      = [stimulus['test1'] for stimulus in t2_trials if 'test1' in stimulus]
image2_stimuli     = [stimulus['image2'] for stimulus in t3_trials if 'image2' in stimulus]
test1_stimuli_t3   = [stimulus['test1'] for stimulus in t3_trials if 'test1' in stimulus]



In [1783]:
def define_t2_trials(stimuli_t1: List[str],stimuli_t2: List[str]) -> List[str]:
    """Sample a single t2 trial"""
    # initialize an empty dictionary to store all variables necessary for a single trial
    trial2 = {}
    for stimulus in range(len(stimuli_t1)):
        # draw a single stimulus for presenting images and testing retrieval
        sampled_stim_t1 = random.choices(stimuli_t1, k=1)
        sampled_stim_t2 = random.choices(stimuli_t2, k=1)
        trial2['image1'] = sampled_stim_t1[0]
        trial2['image2'] = sampled_stim_t2[0]
        trial2['test1'] = sampled_stim_t1[0]
        trial2['test2'] = sampled_stim_t2[0]
        trial2['baselinePresent'] = '0'
        trial2['baselinePresent2'] = '1'
        trial2['adjust2Present'] = '1'
        trial2['trial_type'] = 't2'
        trial2.update(trial2)
    remove_stimuli(stimuli=stimuli_t1, sampled_stimuli=sampled_stim_t1)
    remove_stimuli(stimuli=stimuli_t2, sampled_stimuli=sampled_stim_t2)
    return trial2

def sample_t2_trials(
    stimuli_test1: List[str],
    stimuli_test2: List[str],
    stimulus_order: List[str],
) -> List[str]:
    """Sample stimuli for all t2 trials in a practice or experimental session."""
    sampled_t2_trials = [define_t2_trials(stimuli_t1 = test1_stimuli, stimuli_t2 = test2_stimuli) for stim in stimulus_order]
    return sampled_t2_trials

def define_t3_trials(stimuli_t1: List[str],stimuli_i2: List[str]) -> List[str]:
    """Sample a single t2 trial"""
    # initialize an empty dictionary to store all variables necessary for a single trial
    trial3 = {}
    placeholder = '_'
    for stimulus in range(len(stimuli_t1)):
        # draw a single stimulus for presenting images and testing retrieval
        sampled_stim_t1 = random.choices(stimuli_t1, k=1)
        sampled_stim_i2 = random.choices(stimuli_i2, k=1)
        trial3['image1'] = sampled_stim_t1[0]
        trial3['image2'] = sampled_stim_i2[0]
        trial3['test1'] = sampled_stim_t1[0]
        trial3['test2'] = placeholder
        trial3['baselinePresent'] = '0'
        trial3['baselinePresent2'] = '1'
        trial3['adjust2Present'] = '0'
        trial3['trial_type'] = 't3'
        trial3.update(trial3)
    remove_stimuli(stimuli=stimuli_t1, sampled_stimuli=sampled_stim_t1)
    remove_stimuli(stimuli=stimuli_i2, sampled_stimuli=sampled_stim_i2)
    return trial3

def sample_t3_trials(
    stimuli_test1: List[str],
    stimuli_image2: List[str],
    stimulus_order: List[str],
) -> List[str]:
    """Sample stimuli for all t2 trials in a practice or experimental session."""
    sampled_t3_trials = [define_t3_trials(stimuli_t1 = test1_stimuli_t3, stimuli_i2 = image2_stimuli) for stim in stimulus_order]
    return sampled_t3_trials


In [1784]:
stims = ['stim1','stim2']
stimulus_order = ['stim1','stim2','stim3','stim4','stim5']*4 + stims

t2_trials = sample_t2_trials(stimuli_test1=test1_stimuli,stimuli_test2=test2_stimuli,stimulus_order =stimulus_order)
t3_trials = sample_t3_trials(stimuli_test1=test1_stimuli_t3,stimuli_image2=image2_stimuli,stimulus_order =stimulus_order)
trials = t2_trials + t3_trials + baseline_trials
random.shuffle(trials)
len(stimulus_order)

22

In [1785]:
# convert the list of trials into a pandas dataframe
participant_trials = pd.DataFrame(trials)

In [1786]:
participant_trials["swap"] = np.random.choice(["Yes", "No"], len(participant_trials), p=[0.5, 0.5])
participant_trials

,image1,image2,test1,test2,baselinePresent,baselinePresent2,adjust2Present,trial_type,swap
0,stimuli/acorn.png[56.25],_,stimuli/acorn.png[56.25],_,1,0,0,baseline,Yes
1,stimuli/monument.png[213.75],_,stimuli/monument.png[213.75],_,1,0,0,baseline,No
2,stimuli/lightbulb01.png[146.25],stimuli/fryingpan01.png[168.75],stimuli/lightbulb01.png[146.25],_,0,1,0,t3,Yes
3,stimuli/africanelephant.png[258.75],stimuli/gorilla.png[326.25],stimuli/africanelephant.png[258.75],stimuli/gorilla.png[326.25],0,1,1,t2,No
4,stimuli/backpack01a.png[236.25],_,stimuli/backpack01a.png[236.25],_,1,0,0,baseline,No
...,...,...,...,...,...,...,...,...,...
61,stimuli/hat04a.png[168.75],_,stimuli/hat04a.png[168.75],_,1,0,0,baseline,No
62,stimuli/chipmunk.png[258.75],stimuli/windsurfboard.png[348.75],stimuli/chipmunk.png[258.75],_,0,1,0,t3,Yes
63,stimuli/rhinoceros02.png[33.75],stimuli/nail.png[56.25],stimuli/rhinoceros02.png[33.75],_,0,1,0,t3,No
64,stimuli/graduationcap.png[348.75],stimuli/toytractor02a.png[11.25],stimuli/graduationcap.png[348.75],stimuli/toytractor02a.png[11.25],0,1,1,t2,No


In [1787]:
#participant_trials[['image1','image2']] = pd.np.select(participant_trials['swap'] == 0, participant_trials[['image2','image1']].values, participant_trials[['image1','image2']].values)
participant_trials[['image1','image2']] = participant_trials[['image2','image1']].where((participant_trials['swap'] == 'Yes') & (participant_trials['trial_type'] == 't2'), participant_trials[['image1','image2']].values)
participant_trials[['image1','image2']] = participant_trials[['image2','image1']].where((participant_trials['swap'] == 'Yes') & (participant_trials['trial_type'] == 't3'), participant_trials[['image1','image2']].values)
participant_trials

,image1,image2,test1,test2,baselinePresent,baselinePresent2,adjust2Present,trial_type,swap
0,stimuli/acorn.png[56.25],_,stimuli/acorn.png[56.25],_,1,0,0,baseline,Yes
1,stimuli/monument.png[213.75],_,stimuli/monument.png[213.75],_,1,0,0,baseline,No
2,stimuli/fryingpan01.png[168.75],stimuli/lightbulb01.png[146.25],stimuli/lightbulb01.png[146.25],_,0,1,0,t3,Yes
3,stimuli/africanelephant.png[258.75],stimuli/gorilla.png[326.25],stimuli/africanelephant.png[258.75],stimuli/gorilla.png[326.25],0,1,1,t2,No
4,stimuli/backpack01a.png[236.25],_,stimuli/backpack01a.png[236.25],_,1,0,0,baseline,No
...,...,...,...,...,...,...,...,...,...
61,stimuli/hat04a.png[168.75],_,stimuli/hat04a.png[168.75],_,1,0,0,baseline,No
62,stimuli/windsurfboard.png[348.75],stimuli/chipmunk.png[258.75],stimuli/chipmunk.png[258.75],_,0,1,0,t3,Yes
63,stimuli/rhinoceros02.png[33.75],stimuli/nail.png[56.25],stimuli/rhinoceros02.png[33.75],_,0,1,0,t3,No
64,stimuli/graduationcap.png[348.75],stimuli/toytractor02a.png[11.25],stimuli/graduationcap.png[348.75],stimuli/toytractor02a.png[11.25],0,1,1,t2,No


In [1788]:
# sample 6 practice trials 
size = 2        # sample size
replace = False  # with replacement
fn = lambda obj: obj.loc[np.random.choice(obj.index, size, replace),:]
training = participant_trials.groupby('trial_type', as_index=False).apply(fn)
training = training.sample(frac = 1)

In [1789]:
# create final WM conditions file with 6 trainings trials in the beginning (two of each trial type)
trials_training = participant_trials['image1'].isin(training['image1'])
participant_trials.drop(participant_trials[trials_training].index, inplace = True)
trials_exp = participant_trials
print(len(trials_exp))
print(len(participant_trials))
participant_trials = pd.concat([training,participant_trials.loc[:]]).reset_index(drop=True)
print(len(participant_trials))

60
60
66


In [1790]:
# generate ltm file
trials_exp = pd.DataFrame(trials_exp)
conditions_ltm_image1 = trials_exp[['trial_type','image1',]]
conditions_ltm_image2 = trials_exp[['trial_type','image2',]]
conditions_ltm_image2 = conditions_ltm_image2[conditions_ltm_image2['image2'] != '_']
conditions_ltm_image2 = conditions_ltm_image2.rename(columns={'image2': 'image1'})
conditions_ltm_ = pd.concat([conditions_ltm_image1,conditions_ltm_image2.loc[:]]).reset_index(drop=True)


In [1791]:
conditions_ltm_['startOri_ltm'] = conditions_ltm_['image1'].str.split('[',1).str[1].str.strip()
conditions_ltm_['startOri_ltm'] = conditions_ltm_['startOri_ltm'].str.split(']',0).str[0].str.strip()
conditions_ltm_['image1'] = conditions_ltm_['image1'].str.split('[',0).str[0].str.strip()
conditions_ltm_

,trial_type,image1,startOri_ltm
0,baseline,stimuli/monument.png,213.75
1,t3,stimuli/fryingpan01.png,168.75
2,t2,stimuli/africanelephant.png,258.75
3,baseline,stimuli/backpack01a.png,236.25
4,baseline,stimuli/hen.png,236.25
...,...,...,...
95,t2,stimuli/scissors02a.png,56.25
96,t3,stimuli/chipmunk.png,258.75
97,t3,stimuli/nail.png,56.25
98,t2,stimuli/toytractor02a.png,11.25


In [1792]:
# finish creating WM conditions file
# split orientation and object identity also in working memory experiment
participant_trials['startOri1'] = participant_trials['image1'].str.split('[',1).str[1].str.strip()
participant_trials['startOri1'] = participant_trials['startOri1'].str.split(']',0).str[0].str.strip()
participant_trials['startOri2'] = participant_trials['image2'].str.split('[',1).str[1].str.strip()
participant_trials['startOri2'] = participant_trials['startOri2'].str.split(']',0).str[0].str.strip()
participant_trials['image1'] = participant_trials['image1'].str.split('[',0).str[0].str.strip()
participant_trials['image2'] = participant_trials['image2'].str.split('[',0).str[0].str.strip()
participant_trials['test1'] = participant_trials['test1'].str.split('[',0).str[0].str.strip()
participant_trials['test2'] = participant_trials['test2'].str.split('[',0).str[0].str.strip()

In [1793]:
# converting string (numbers) to datatype float
participant_trials['startOri1'] = participant_trials['startOri1'].astype(float)
participant_trials['startOri2'] = participant_trials['startOri2'].astype(float)

In [1794]:
# check that the presentation orientation of the two stimuli is not the same
participant_trials['orientation_match'] = np.where(participant_trials['startOri1'] == participant_trials['startOri2'], 'True', 'False')

found = participant_trials[participant_trials['orientation_match'].str.contains('True')]
print(found.count())

image1               2
image2               2
test1                2
test2                2
baselinePresent      2
baselinePresent2     2
adjust2Present       2
trial_type           2
swap                 2
startOri1            2
startOri2            2
orientation_match    2
dtype: int64


In [1795]:
# cleaning up dataframe regarding missing values etc. that psychopy usually does not tolerate in condition files
values = "_"
participant_trials['startOri2'] = participant_trials['startOri2'].fillna(value = values) 
participant_trials

,image1,image2,test1,test2,baselinePresent,baselinePresent2,adjust2Present,trial_type,swap,startOri1,startOri2,orientation_match
0,stimuli/gecko.png,stimuli/pintofbeer.png,stimuli/pintofbeer.png,_,0,1,0,t3,Yes,326.25,281.25,False
1,stimuli/wateringcan.png,stimuli/duck01.png,stimuli/wateringcan.png,stimuli/duck01.png,0,1,1,t2,No,258.75,326.25,False
2,stimuli/cellphone.png,stimuli/motorcycle.png,stimuli/cellphone.png,_,0,1,0,t3,No,78.75,146.25,False
3,stimuli/mailbox02.png,_,stimuli/mailbox02.png,_,1,0,0,baseline,No,101.25,_,False
4,stimuli/desktopcomputer.png,stimuli/pitcher02a.png,stimuli/pitcher02a.png,stimuli/desktopcomputer.png,0,1,1,t2,Yes,281.25,101.25,False
...,...,...,...,...,...,...,...,...,...,...,...,...
61,stimuli/hat04a.png,_,stimuli/hat04a.png,_,1,0,0,baseline,No,168.75,_,False
62,stimuli/windsurfboard.png,stimuli/chipmunk.png,stimuli/chipmunk.png,_,0,1,0,t3,Yes,348.75,258.75,False
63,stimuli/rhinoceros02.png,stimuli/nail.png,stimuli/rhinoceros02.png,_,0,1,0,t3,No,33.75,56.25,False
64,stimuli/graduationcap.png,stimuli/toytractor02a.png,stimuli/graduationcap.png,stimuli/toytractor02a.png,0,1,1,t2,No,348.75,11.25,False


In [1796]:
# export my excel files for the working memory and ltm memory task
file_name_WM = 'conditions_5.xlsx'
participant_trials.to_excel(file_name_WM, index=False)

file_name_LTM = 'conditions_ltm_5.xlsx'
conditions_ltm_.to_excel(file_name_LTM, index=False)